In [1]:
from neo4j_utility import *
from llm_extraction import *
from firecrawl_scraping import *
from utility import *
import os
from tqdm import tqdm
from dotenv import load_dotenv

/Users/chenkangan/Documents/UCL DSML/Thesis/project/ucl_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import dotenv
import os
from neo4j import GraphDatabase


URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_INSTANCE_PASSWORD"))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")


Connection established.


In [12]:
doc_list = os.listdir('extraction_output_v2')
for doc in doc_list[:20]:
    processed_name = doc.replace('_extraction.json', '')
    
    extraction_file_path = f'extraction_output_v2/{processed_name}_extraction.json'
    
    kg_construction(processed_name, extraction_file_path)

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 61] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection refused)

In [10]:
data = read_json_file('data/client_info.json')

1572

In [ ]:
doclist = os.listdir('scraping_output_v2_raw')
for doc in tqdm(doclist):
    try:
        processed_name = doc.replace('.json', '')
        scrape_file_path = f'scraping_output_v2_raw/{processed_name}.json'
        summary_file_path = f'extraction_summary_v2/{processed_name}_summary_str.json'
        extraction_file_path = f'extraction_output_v2/{processed_name}_extraction.json'

        _ = llm_summary_execution(processed_name = processed_name,
                            scrape_file_path = scrape_file_path,
                            summary_file_path = summary_file_path,
                            model_name = 'gpt-4o-mini',
                            overwrite = False)

        _ = llm_extraction_execution(processed_name = processed_name,
                                        summary_file_path = summary_file_path,
                                        extraction_file_path = extraction_file_path, 
                                        include_additional_context = True, 
                                        model_name = 'gpt-4o',
                                        overwrite = False)

        _ = get_product_embedding(processed_name = processed_name,
                            extraction_file_path = extraction_file_path)

        _ = add_client_url_to_extraction_output(processed_name = processed_name,
                                    extraction_file_path = extraction_file_path)

        _ = update_client_list(processed_name = processed_name,
                        extraction_file_path = extraction_file_path,
                        client_file_path = 'data/client_info.json')
    except Exception as e:
        print(f'Error at {processed_name}: {e}')
        

In [ ]:
doclist = os.listdir('scraping_output_v2_raw')
for doc in tqdm(doclist):
    
    try:
        processed_name = doc.replace('.json', '')
        scrape_file_path = f'scraping_output_v2_raw/{processed_name}.json'
        summary_file_path = f'extraction_summary_v2/{processed_name}_summary_str.json'
        
        _ = llm_summary_execution(processed_name = processed_name,
                        scrape_file_path = scrape_file_path,
                        summary_file_path = summary_file_path,
                        model_name= 'gpt-4o-mini',
                        overwrite = False)
        
    except Exception as e:
        print(f'Error at {processed_name}: {e}')
    

In [14]:
processed_name = 'crunch_'
    
scrape_file_path = f'scraping_output_v2_raw/{processed_name}.json'
summary_file_path = f'extraction_summary_v2/{processed_name}_summary_str.json'
extraction_file_path = f'extraction_output_v2/{processed_name}_extraction.json'

_ = llm_summary_execution(processed_name = processed_name,
                      scrape_file_path = scrape_file_path,
                      summary_file_path = summary_file_path,
                      model_name = 'gpt-4o-mini',
                      overwrite = True)

_ = llm_extraction_execution(processed_name = processed_name,
                                summary_file_path = summary_file_path,
                                extraction_file_path = extraction_file_path, 
                                include_additional_context = True, 
                                overwrite = True)

_ = get_product_embedding(processed_name = processed_name,
                    extraction_file_path = extraction_file_path)

_ = add_client_url_to_extraction_output(processed_name = processed_name,
                            extraction_file_path = extraction_file_path)

_ = update_client_list(processed_name = processed_name,
                extraction_file_path = extraction_file_path,
                client_file_path = 'data/client_info.json')

Company: crunch_; Content in /partners is extracted.
Company: crunch_; Content in main_page is extracted.
Company: crunch_; Information extraction begins.
Company: crunch_; Estimated Cost: $0.003855
Company: crunch_; Pitchbook description obtained: Developer of financial business management software designed to serve organizations with all their accounting needs. The company's software is designed for the business owner and includes features like real-time dashboards to get an immediate overview of how the business is performing, enabling them to get the absolute most out of running their own business.
Company: crunch_; PART 1 - Initial extraction is completed.
Company: crunch_; PART 2 - Skipped, due to lack of client information.
Company: crunch_; Embedding is completed.
Company: crunch_; No clients' information.
Company: crunch_; No clients to be updated


Issue page:
- zenefits

In [ ]:

def inspect_page(processed_name, endpoint = 'main_page'):
   
    scraping_data = read_json_file(f'scraping_output_v2_raw/{processed_name}.json')
    print('---------------------------------------------')
    print(f'Before cleaning: {endpoint}')
    print('---------------------------------------------')
    print(scraping_data[endpoint])
    
    print('---------------------------------------------')
    print(f'After cleaning: {endpoint}')
    print('---------------------------------------------')
    print(clean_scraped_content(scraping_data[endpoint]))
    
    print('---------------------------------------------')
    print(f'Summary of page: {endpoint}')
    print('---------------------------------------------')
    summary_data = read_json_file(f'extraction_summary_v2/{processed_name}_summary_str.json')
    print(summary_data[endpoint])

 

In [ ]:
inspect_page('truelytics')

---------------------------------------------
Before cleaning: main_page
---------------------------------------------
[![truelytics-envestnet-logo-white](https://www.truelytics.com/hubfs/truelytics-envestnet-logo-white.svg "truelytics-envestnet-logo-white")](https://www.truelytics.com)

*   [SOFTWARE](https://www.truelytics.com/software)
    *   [PLATFORM OVERVIEW](https://www.truelytics.com/software)
        *   [TrueRecruit](https://www.truelytics.com/truerecruit)
            
        *   [TruePerformance](https://www.truelytics.com/trueperformance)
            
        *   [TrueContinuity](https://www.truelytics.com/truecontinuity)
            
        *   [TrueMatch](https://www.truelytics.com/truematch)
            
    *   [NOTABLE FEATURES](https://www.truelytics.com/software)
        *   [Data & Benchmarking](https://www.truelytics.com/data-and-benchmarking)
            
        *   [Enterprise Functionality](https://www.truelytics.com/enterprise-functionality)
            
  

In [ ]:

processed_name = 'reveel'
summary_data = read_json_file(f'extraction_summary_v2/{processed_name}_summary_str.json')

summary_data

{'main_page': '"Reveel is your Onchain Account for the new internet. Participate & send money to anyone, anywhere, on any chain. Gain points for all your onchain activity. No matter which wallet you use, if it’s connected to your Reveel account, your transactions generate rewards. Reveel is your multi-chain & multi-wallet account. You can finally aggregate your onchain activity. Import all your wallets. Connect to your bank, move money on & off-chain seamlessly, and send whichever token - stablecoin or memecoin - you chose. Connect to dApps with any of your wallets & participate in the new internet. Reveel will even bring the onchain economy to you (coming soon). Our Revenue Share platform helps leading brands & creators automate their accounting & distributing profits to partners."',
 'timestamp': '2024-08-01 20:50 Etc/GMT',
 'processed_company': 'reveel',
 'url': 'https://www.r3vl.xyz',
 'model_name': 'gpt-4o-mini'}

In [ ]:
summary_data

{'/solutions/': '"Bento currently offers 5 powerful programs that allow you to serve all of your clients and prospects comprehensively during Moments That Matter.\n5 programs for you to systematically lead with advice — for everyone’s benefit.\nWant to proactively advise your entire book of business on upcoming age milestones that trigger distinct wealth management risks or opportunities? You can do that.\nWant to be there for your clients during major Life Events such as getting married, divorced or having a child? We\'ve got you covered.\nCare about the Next Generation of your client family and want to help parents instill Financial Literacy? Bento can help.\nHave a large prospect book or cover thousands of Retirement Plan Participants and want to cover them automatically with impactful guidance at scale? Our Bento Direct program can do that for you.\nHave a website? Want to book more prospect meetings? Bento can help, by leading with relevant advice.\nBento connects via APIs into yo

In [ ]:
doc_list = os.listdir('client_extraction_summary')
for doc in doc_list:
    
    try: 
        data = read_json_file(f'client_extraction_summary/{doc}')
        # if 'timestamp' not in data:
        #     current_dateTime = datetime.now(pytz.timezone('Etc/GMT'))
        #     data['timestamp'] = current_dateTime.strftime(format = "%Y-%m-%d %H:%M")
            
        # if "processed_company" not in data:
        #     process_name = doc.replace('.json', '')
        #     data["processed_company"] = process_name
            
        # if "url" not in data:
        #     data["url"] = "https://" + get_additional_info(process_name, 'processed_url')
            
        if "model_name" not in data:
            data['model_name'] = 'gpt-4o-mini'
        
        write_json_file(f'client_extraction_summary/{doc}', data)
    except Exception as e:
        print(f'Error occurs on {doc}: {e}')